In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [ ]:
import os

import numpy as np
import rioxarray
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

In [ ]:
# product definition 
product_folder = (
    "data/S1A_IW_SLC__1SDV_20211223T170557_20211223T170624_041139_04E360_B8E2.SAFE"
)
dem_urlpath_10 = "data/Gran-Sasso-10m-DEM.tif"
measurement_group = "IW3/VV"
dem_10 = scene.open_dem_raster(dem_urlpath_10)

In [ ]:
dem_10.plot();

### GTC

In [ ]:
gtc_path = os.path.basename(product_folder) + ".10m.GTC.tif"
apps.backward_geocode_sentinel1(
    product_folder, 
    measurement_group, 
    dem_urlpath_10, 
    output_urlpath=gtc_path, 
)
gtc = xr.open_dataarray(gtc_path, cache=False).drop("band");

In [ ]:
gtc.plot(vmax=0.5);

### RCT

In [ ]:
rtc_gamma_10m_path = os.path.basename(product_folder) + ".10m.RTC.gamma.tif"
apps.backward_geocode_sentinel1(
    product_folder, 
    measurement_group, 
    dem_urlpath_10, 
    interp_method="nearest",
    correct_radiometry="gamma",
    output_urlpath=rtc_gamma_10m_path, 
    grouping_area_factor=(3, 13),
)
rtc_gamma_10m  = rioxarray.open_rasterio(rtc_gamma_10m_path, cache=False, chunks=-1)

In [ ]:
rtc_gamma_10m.plot(vmax=0.0005);

### Comparison GTC RTC

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 15))

gtc.plot(ax=axes[0], vmax=0.5);
axes[0].grid(c="red")

rtc_gamma_10m.plot(ax=axes[1], vmax=0.0005);
axes[1].grid(c="red")